# Account Routes

In [ ]:
#| default_exp routes.account

In [ ]:
#| exporti
from typing import Union
import httpx
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [ ]:
#| export
async def get_accounts(auth: dmda.DomoAuth,
                       debug_api: bool = False, 
                       session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None):
    """retrieve a list of all the accounts the user has read access to.  Note users with "Manage all accounts" will retrieve all account objects"""
    
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts"

    return await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

#### sample implementation of get_accounts

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)


res = await get_accounts(auth = token_auth)
pd.DataFrame(res.response)

,id,userId,name,displayName,type,valid,dataProviderType,credentialsType,createdAt,createdBy,modifiedAt,modifiedBy,configurations,accountTemplateId,accountTemplateAuthorizationId,accountId
0,1,1893952720,DataSet Copy Account,DataSet Copy Account,data,True,dataset-copy,fields,1589100087000,1893952720,1589100087000,1893952720,{},None,None,1
1,5,1893952720,Domo Governance Datasets Third Party Account,test_rename,data,True,domo-governance-d14c2fef-49a8-4898-8ddd-f64998...,fields,1616777681000,1893952720,1674849589000,1893952720,{},None,None,5
2,27,1893952720,DataSet Copy Account,DataSet Copy Account,data,True,domo-csv,fields,1619083568000,1893952720,1619083568000,1893952720,{},None,None,27
3,45,1893952720,onyxReporting@gmail.com,onyxReporting@gmail.com,data,True,google-spreadsheets,oauth,1664924354000,1893952720,1664924354000,1893952720,{},None,None,45


In [ ]:
#| export
async def get_account_from_id(auth: dmda.DomoAuth, account_id: int,
                              debug_api: bool = False, session: httpx.AsyncClient = None):
    """retrieves metadata about an account"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}?unmask=true"

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

#### sample implementation of get_account_from_id

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)


res = await get_account_from_id(auth=token_auth, account_id = 1)
res.response


{'id': 1,
 'userId': 1893952720,
 'name': 'DataSet Copy Account',
 'displayName': 'DataSet Copy Account',
 'type': 'data',
 'valid': True,
 'dataProviderType': 'dataset-copy',
 'credentialsType': 'fields',
 'createdAt': 1589100087000,
 'createdBy': 1893952720,
 'modifiedAt': 1589100087000,
 'modifiedBy': 1893952720,
 'configurations': {},
 'accountTemplateId': None,
 'accountTemplateAuthorizationId': None,
 'accountId': 1}

# Account Config

In [ ]:
#| export
class AccountConfig_InvalidDataProvider(Exception):
    def __init__(self, account_id:str, data_provider_type:str, domo_instance: str):
        message = f"Account - {account_id}, could not be retrieved with data_provider_type, '{data_provider_type}' from {domo_instance}"
        super().__init__( message)

async def get_account_config(auth: dmda.DomoAuth,
                             account_id: int,
                             data_provider_type: str ,
                             debug_api: bool = False, 
                             session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None):

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/providers/{data_provider_type}/account/{account_id}?unmask=true"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

    if res.response == {}:
        raise AccountConfig_InvalidDataProvider(account_id= account_id, data_provider_type= data_provider_type, domo_instance=auth.domo_instance)
    
    return res

#### sample implementation of get_account_config

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_account_config(auth=token_auth, account_id=1, data_provider_type='dataset-copy')
res.response


{'instance': 'northshore-io-partner.domo.com', 'accessToken': '********'}

In [ ]:
#| export
async def update_account_config(auth: dmda.DomoAuth,
                                account_id: int,
                                config_body: dict,
                                data_provider_type: str,
                                debug_api: bool = False, 
                                session: httpx.AsyncClient = None):

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/providers/{data_provider_type}/account/{account_id}"

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=config_body,
        debug_api=debug_api,
        session=session
    )

In [ ]:
#| export
async def update_account_name(auth: dmda.DomoAuth,
                              account_id: int,
                              account_name: str,
                              debug_api: bool = False, 
                              session: httpx.AsyncClient = None):

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}/name"
    
    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=account_name,
        content_type = "text/plain",
        debug_api=debug_api,
        session=session
    )

In [ ]:
#| export
async def create_account(auth:dmda.DomoAuth, config_body:dict,
                         debug_api: bool = False, session: httpx.AsyncClient = None):

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts"

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='POST',
        body = config_body,
        debug_api=debug_api,
        session=session
    )

In [ ]:
#| export
async def delete_account(auth:dmda.DomoAuth,
                         account_id: str,
                         debug_api: bool = False, 
                         session: httpx.AsyncClient = None):
    
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}"

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='DELETE',
        debug_api=debug_api,
        session=session
    )

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
